In [8]:
from arch import arch_model
import pandas as pd
from pandas.plotting import autocorrelation_plot
from sktime.forecasting.all import temporal_train_test_split
from sktime.forecasting.all import ExponentialSmoothing
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.forecasting.all import ForecastingHorizon
from sklearn.metrics import mean_squared_error, mean_absolute_error
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train_msft.csv',index_col = 0)
train.index = pd.to_datetime(train.index)

val = pd.read_csv('val_msft.csv',index_col = 0)
val.index = pd.to_datetime(val.index)

test = pd.read_csv('test_msft.csv',index_col = 0)
test.index = pd.to_datetime(test.index)

# select model on train based on AIC

In [15]:
def fit_garch_model(train_data, p, q):
    model = arch_model(train_data, vol='Garch', p=p, q=q)
    model_fit = model.fit(disp='off')
    return model_fit.aic, model_fit

# Grid search for best GARCH model based on AIC
best_aic = float('inf')
best_model = None
best_pq = None

for p in range(1, 6):
    for q in range(1, 6):
        aic, model_fit = fit_garch_model(train['adj_close'], p, q)
        if aic < best_aic:
            best_aic = aic
            best_model = model_fit
            best_pq = (p, q)
best_pq

(1, 5)

#  Fit best model on validation data

In [61]:
forecast = best_model.forecast(horizon=len(val))
forecast = pd.Series(forecast.variance.values[-1,:], index=val.index)

In [59]:
def get_evaluation_simple(forecast,test):
    
    # Calculate the error metrics
    mae = mean_absolute_error(test, forecast)
    mse = mean_squared_error(test, forecast)
    mape = np.mean(np.abs((test - forecast) / test))
    
    # Calculate sMAPE
    smape = np.mean(2*np.abs(forecast - test) / ((np.abs(test) + np.abs(forecast))))

    df_metrics = pd.DataFrame({
        'MAE': [mae],
        'MSE': [mse],
        'MAPE': [mape],
        'sMAPE': [smape]
    })

    return df_metrics

In [60]:
get_evaluation_simple(forecast,val['adj_close'])

,MAE,MSE,MAPE,sMAPE
0,187.654448,35374.186366,3.505977,1.227595
